In [ ]:
# ==== GPU SETUP CELL (Added) ====
import os, torch

# optional: restrict visible GPU(s)
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# enforce CUDA
assert torch.cuda.is_available(), "❌ CUDA not available. Please check your GPU setup."

device = torch.device("cuda")
print("✅ Using device:", device)
print("GPU:", torch.cuda.get_device_name(0))

# cuDNN speedup settings
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True


import Libraries


In [ ]:
import warnings
warnings.filterwarnings('ignore') 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from PIL import Image
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torch.optim as optim
import torch.nn.functional as F
import colorama
from colorama import Fore, Style

import dataset

In [ ]:
Root_dir = "D:/DhanshreeandTeamAI/Github/plant_disease_detection/dataset/New Plant Diseases Dataset(Augmented)"
train_dir = Root_dir + "/train"
valid_dir = Root_dir + "/valid"
test_dir = "D:/DhanshreeandTeamAI/Github/plant_disease_detection/dataset/test"
Diseases_classes = os.listdir(train_dir)

#### Data Preprocessing

How many classes in the dataset? 

Name of classes and Numbr them ?

In [ ]:
print(Fore.GREEN +str(Diseases_classes))
print("\nTotal number of classes are: ", len(Diseases_classes))


  - How many image are in each classes?

In [ ]:
plt.figure(figsize=(60,60), dpi=200)
cnt = 0
plant_names = []
tot_images = 0

for i in Diseases_classes:
    cnt += 1
    plant_names.append(i)
    plt.subplot(7,7,cnt)
    
    image_path = os.listdir(train_dir + "/" + i)
    print(Fore.GREEN)
    print("The Number of Images in " +i+ ":", len(image_path), end= " ")
    tot_images += len(image_path)
    
    img_show = plt.imread(train_dir + "/" + i + "/" + image_path[0])
    
    plt.imshow(img_show)
    plt.xlabel(i,fontsize=30)
    plt.xticks([])
    plt.yticks([])
    
    
print("\nTotal Number of Images in Directory: ", tot_images)

Comparing the Number of Classes**

  - It's really important to know which classes have the most images and which have the lowest.

In [ ]:
plant_names = []
Len = []
for i in Diseases_classes:
    plant_names.append(i)
    imgs_path = os.listdir(train_dir + "/" + i)
    Len.append(len(imgs_path))

Len.sort(reverse=True)

sns.set(style="whitegrid", color_codes=True)
plt.figure(figsize=(20,20),dpi=200)
ax = sns.barplot(x= Len, y= plant_names, palette="Greens")
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.show()




* **ImageFolder/ ToTensor**

  - The ImageFolder class in PyTorch, part of the Torchvision library, is a versatile data loader specifically designed for handling image datasets. It simplifies the process of loading and organizing image data for training machine learning models. Let’s delve into its details:
Purpose: The ImageFolder class is used to load image data from a directory structure where images are organized into subfolders representing different classes or categories. By default, it assumes the following directory structure:

<div style = 'border : 3px solid non; background-color:#ECFFDC ; ;padding:10px'>

    root/
    ├── class_1/
    │   ├── image1.jpg
    │   ├── image2.jpg
    │   └── ...
    ├── class_2/
    │   ├── image3.jpg
    │   ├── image4.jpg
    │   └── ...
    └── ...


In [ ]:
train = ImageFolder(train_dir, transform=transforms.ToTensor())
valid = ImageFolder(valid_dir, transform=transforms.ToTensor()) 

In [ ]:
train


In [ ]:
train[0]


The last number (0, 1, 2, ...) shows the number of classes

In [ ]:
train[7000]

In [ ]:
train[70000]

In [ ]:
img, label = train[0]
print(img.shape, label)

3 is the number of channels (RGB) and 256 x 256 is the width and height of the image

In [ ]:
def show_image(image, label):
    print("Label :" + train.classes[label] + "(" + str(label) + ")")
    plt.imshow(image.permute(1, 2, 0))
    
    
image_list = [0, 3000, 5000, 8000, 12000, 15000, 60000, 70000]
    
chs = 0
for img in image_list:
    chs += 1
    plt.subplot(2,4,chs)
    print(Fore.GREEN)
    plt.tight_layout()
    plt.xlabel(img,fontsize=10)
    plt.title(train[img][1])
    show_image(*train[img])

In [ ]:
batch_size = 32

**DataLoader:**
   - Dataloader is a class for PyTorch data loading utility. It is used to import data from datasets. 
    
   - **Dataloader has two different types of datasets:** map-style datasets and iterable-style datasets. The PyTorch DataLoader class is an important tool to help you prepare, manage, and serve your data to your deep learning networks.

In [ ]:
# DataLoaders for training and validation
train_dataloader = DataLoader(train, batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_dataloader = DataLoader(valid, batch_size, num_workers=2, pin_memory=True)

### Device of Process

**GPU/CPU:** 
   - When designing your deep learning architecture, your decision to include GPUs relies on several factors:

   - Memory bandwidth—including GPUs can provide the bandwidth needed to accommodate large datasets. 
    
   - Dataset size GPUs in parallel can scale more easily than CPUs, enabling you to process massive datasets faster.
    
   - Optimization a downside of GPUs is that optimization of long-running individual tasks is sometimes more difficult than with CPUs.

In [ ]:
# for moving data into GPU (if available)
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available:
        return torch.device("cuda")
    else:
        return torch.device("cuda") 

<div style = 'border : 3px solid non; background-color:#ECFFDC ; ;padding:10px'>


**Data to Device:** 


   - In PyTorch, it’s crucial to use .to(device) when you explicitly want to move tensors (data) or entire models (including layers and parameters) to a specific device for computation. **Here’s why:**

   - **Consistent Device Placement:**
When working with GPUs (Graphics Processing Units), you need to ensure that both the model and the data reside on the same device (either CPU or GPU).
If the data is on the CPU and the model is on the GPU (or vice versa), you’ll encounter a runtime error.
To avoid this, use .to(device) to transfer both the data and the model to the same device.
    
   - **Avoiding Mixed Device Operations:**
If an operation involves one tensor on the GPU and another on the CPU, PyTorch will raise a Runtime Error.
For example, attempting to perform operations between tensors on different devices will result in an error like: “Expected object of device type cuda but got device type cpu.”
    
   - **Setting the Device:**
You can set a variable device to 'cuda' if a GPU is available, otherwise set it to 'cpu'.
Then, move both the model and the data to the specified device:

In [ ]:
# for moving data to device (CPU or GPU)
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [ ]:
# for loading in the device (GPU if available else CPU)
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dataloader, device):
        self.dataloader = dataloader
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dataloader:
            yield to_device(b, self.device)
        
    def __len__(self):
        """Number of batches"""
        return len(self.dataloader)

In [ ]:
device = get_default_device()
device

- CUDA is a programming model and computing toolkit developed by NVIDIA. It enables you to perform compute-intensive operations faster by parallelizing tasks across GPUs. CUDA is the dominant API used for deep learning although other options are available, such as OpenCL. PyTorch provides support for CUDA in the torch.cuda library.

In [ ]:
# Moving data into GPU, WrappedDataLoader
train_dataloader = DeviceDataLoader(train_dataloader, device)
valid_dataloader = DeviceDataLoader(valid_dataloader, device)

### CNN(Convolutional Neural Network)


**Function of Acc:** 


   - We can compare the outputs of model with the true labels.

In [ ]:
# for calculating the accuracy
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

**Classification Base:** 


   - Let's create a step by step classification base module:
    
   - **First, Training_step:** the images and labels take values from batch. The output of model is a type of images and the loss is calculate from **F** cross entropy function (out, labels). It seems that the prediction is compared with the actual values in labels.
    
   - **Second, validation step:** is just like the above one. But it has another attribute call acc.
    
   - **Third, validation_epoch_end:** calculate the losses and acces of each batchs and epochs.
    
   - **Fourth, epoch_end:** That shows the final results of everything.

In [ ]:
class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

**ConvBlock:** 


   - Number of inputs, outputs, kernel size and padding are here.
    
   - Also we have Batchnomalization

In [ ]:
# convolution block with BatchNormalization
def ConvBlock(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
             nn.BatchNorm2d(out_channels),
             nn.ReLU(inplace=True)]
    if pool:
        layers.append(nn.MaxPool2d(4))
    return nn.Sequential(*layers)

**CNN:** 


   - And the CNN part
    
   - We can use the class of **ImageClassificationBase** for taking methods like calculating the loss and acc of epochs. And after that, we can set our layers. We can use more layers but in this case, the size of images don't allow to do that **Going to be zero**. In the second part, we have to connect layers to each other. The output of layer number **N**, is the input of layer number **N+1**. And so on.

In [ ]:
# resnet architecture 
class CNN_NeuralNet(ImageClassificationBase):
    def __init__(self, in_channels, num_diseases):
        super().__init__()
        
        self.conv1 = ConvBlock(in_channels, 64)
        self.conv2 = ConvBlock(64, 128, pool=True) 
        self.res1 = nn.Sequential(ConvBlock(128, 128), ConvBlock(128, 128))
        
        self.conv3 = ConvBlock(128, 256, pool=True) 
        self.conv4 = ConvBlock(256, 512, pool=True)
        #self.conv5 = ConvBlock(256, 256, pool=True)
        #self.conv6 = ConvBlock(256, 512, pool=True)
        #self.conv7 = ConvBlock(512, 512, pool=True)
        
        self.res2 = nn.Sequential(ConvBlock(512, 512), ConvBlock(512, 512))
        self.classifier = nn.Sequential(nn.MaxPool2d(4),
                                       nn.Flatten(),
                                       nn.Linear(512, num_diseases))
        
    def forward(self, x): # x is the loaded batch
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        #out = self.conv5(out)
        #out = self.conv6(out)
        #out = self.conv7(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        return out        

In [ ]:
import sys
print(sys.executable)


In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))


In [ ]:
#Test CUDA first:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU name: {torch.cuda.get_device_name(0)}")

#### Model:


   - Now it's time to connet the model to_device for using GPU

In [ ]:
""" # defining the model and moving it to the GPU
# 3 is number of channels RGB, len(train.classes()) is number of diseases.
model = to_device(CNN_NeuralNet(3, len(train.classes)), device) 
model
 """



import torch

# Check and activate GPU
device = torch.device("cuda")
print(f"Using device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("GPU not available, using CPU")

# defining the model and moving it to GPU
# 3 is number of channels RGB, len(train.classes()) is number of diseases.
model = to_device(CNN_NeuralNet(3, len(train.classes)), device) 
model




In [ ]:
# Test GPU activation
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Current device: {next(model.parameters()).device}")